# Packages

In [1]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Open Nighttime Lights'

# Background on VIIRS-DNB monthly composites

Temporal composites, such as annual composites, are created for satellite data for a variety of reasons. Sometimes it can be desirable to visualize or conduct trend analysis on a smoother, less dense time series, especially if you’re comparing multiple data sources and need a common temporal unit of analysis, like a year.

Often the aim is to reduce the noise that occurs with shorter time periods under the intuition that noise (e.g. via stochastic processes) will be minimalized, or “canceled out”, when data are aggregated over longer time periods, whereas the true signal will be preserved or even strengthened relative to the noise levels.

Here, we’re going to work with the VIIRS-Day/Night Band (DNB) image collection that has been corrected for stray-light and filtered for data quality, which includes cloud coverage. The daily images are aggregated into monthly composites. In Google Earth Engine, this Image Collection is: `NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG`

Note: these monthly composites are not filtered to screen out light from auroras, fires, boats or other temporal light.

Cleaning and creating composites using daily Sensor Data Records made available through the partnership with the World Bank, University of Michigan, and NOAA will also be the subject of a more advanced Python library as part of this Open Night Lights platform.

# Get ImageCollection for 2015 VIIRS-DNB data

In [3]:
# Get the 2015 image collection, we're using the "avg_rad" band
viirs2015 = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG')\
            .filterDate('2015-01-01', '2015-12-31')\
            .select('avg_rad')

# There should be 12 images in this collection
print(f'There are {viirs2015.size().getInfo()} images in this collection')

There are 12 images in this collection


# Create an annual composite using the median of these images

As the names suggest, reducer functions like `median` and `mean` reduce a given ImageCollection to a single Image by calculating the median or the mean.  We’ll reduce our ImageCollection of 12 images from 2015 to a single Image by calculating the median. Then we’ll initiate a geemap Map object and visualize our layer as we’ve done before.

As we did with our DMSP-OLS images, let's also apply the mask to ignore cells in our raster that contain no data.

In [4]:
viirs2015med = viirs2015.median()

# Initialize map on Sao Paulo, Brazil
lat = -23.54
lon = -46.63

# Initialize map
map1 = geemap.Map(center = [lat, lon], zoom = 8)
map1.add_basemap('SATELLITE')
map1.addLayer(viirs2015med.mask(viirs2015med), {}, 'VIIRS-DNB 2015 (monthly median)')
map1.addLayerControl()
map1

Map(center=[-23.54, -46.63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

# Create a time series of annual composites from 2015-2019

If you want to create a time series of annual composites, you can filter and reduce the collection as we did for 2015, but for each year you want data.

First, use EE's `list` method to create a list object of years.

In [17]:
start = 2015
end = 2019

years = ee.List.sequence(start, end)
print(f'Our list has {years.size().getInfo()} years in it')

Our list has 5 years in it


Create a function to filter Images to a given year and reduce them, producing an annual composite that we'll map to each year in our list and create a new image collection.

In [20]:
colID = "NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG"

def viirs_annual_median_reduce(year):
    return ee.ImageCollection(colID).filter(
        ee.Filter.calendarRange(year,year,"year")).select("avg_rad").median().set('year',year)

# map function to each year in our list
yearComps = ee.ImageCollection.fromImages(years.map(viirs_annual_median_reduce))
yearComps

Now filter on our image collection on any year to get that composite. We could also add multiple years to our map objects for comparison.

In [21]:
map2 = geemap.Map(center = [lat, lon], zoom = 8)
map2.add_basemap('SATELLITE')

for year in range(start, end + 1):
    img = yearComps.filterMetadata("year", 'equals', year).first() # There's only one image, but we extract from collection
    map2.addLayer(img.mask(img), {}, f'VIIRS-DNB {year}', opacity = 0.75)

map2.addLayerControl()
map2

Map(center=[-23.54, -46.63], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [22]:
# Create a split panel map
left_layer = geemap.ee_tile_layer(yearComps.filterMetadata('year', 'equals', 2015), {}, 
                                  'VIIRS-DNB 2015', opacity = 0.75)
right_layer = geemap.ee_tile_layer(yearComps.filterMetadata('year', 'equals', 2019), {},
                                   'VIIRS-DNB 2019', opacity = 0.75)

map3 = geemap.Map(center = [lat, lon], zoom = 8)
map3.add_basemap('SATELLITE')
map3.split_map(left_layer = left_layer, right_layer = right_layer)
map3.addLayerControl()
map3

Map(center=[-23.54, -46.63], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

VIIRS-DNB has noticeably higher resolution than the DMSP-OLS, so differences in light spatial distribution are visible when investigating dynamics over even a few years.